In [27]:
import tensorflow as tf
import tensorflow_datasets as tfds


#Descargar set de datos de MINIST (Numeros escritos a mano, etiquetados)
datos, metadatos = tfds.load('mnist', as_supervised=True, with_info=True)

#Obtenemos en variables separados los datos de entrenamiento (60k) y pruebas (10k)
datos_entrenamiento, datos_pruebas=datos['train'], datos['test']

#Etiquetas de las 10 categorias posibles (simplemente son los números del 0 al 9)
nombres_clases=metadatos.features['label'].names

#Funcion de normalizacion para los datos (Pasar valor de los pixeles de 0-255 a 0-1)
#Hace que la red aprenda mejor y mas rapido
def normalizar(imagenes, etiquetas):
   imagenes=tf.cast(imagenes, tf.float32)
   imagenes /=255 #Aqui lo pasa de 0-255 a 0-1
   return imagenes, etiquetas

#Normalizar los datos de entrenamiento y pruenas con la funcion que hicimos
datos_entrenamiento=datos_entrenamiento.map(normalizar)
datos_pruebas=datos_pruebas.map(normalizar)

#Agregar a cache (usar memoria en lugar de disco, entrenamiento mas rapido)
datos_entrenamiento=datos_entrenamiento.cache()
datos_pruebas=datos_pruebas.cache()

#crear el modelo
modelo=tf.keras.Sequential([
	tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28,1)),
	tf.keras.layers.MaxPooling2D(2, 2),


        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
	tf.keras.layers.MaxPooling2D(2, 2),

        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
	tf.keras.layers.MaxPooling2D(2, 2),

        tf.keras.layers.Dropout(0.5),
	tf.keras.layers.Flatten(),
	tf.keras.layers.Dense(512, activation='relu'),
	tf.keras.layers.Dense(10, activation=tf.nn.softmax) #Para redes de clasificacion
        ])


#Compilar el modelo
modelo.compile(
	optimizer='adam',
	loss=tf.keras.losses.SparseCategoricalCrossentropy(),
	metrics=['accuracy'])

#Los numeros de datos en entrenamiento y pruebas (60k y 10k)
num_ej_entrenamiento=metadatos.splits["train"].num_examples
num_ej_pruebas=metadatos.splits["test"].num_examples

#El trabajo por lotes permite que entrenamientos con gran cantidad de datos se haga de mas eficiente
TAMANO_LOTE=32

#Shufle y repeat hacen que los datos esten mezclados de manera aleatoria para que la red
#no se vaya a aprender el orden de las cosas
datos_entrenamiento=datos_entrenamiento.repeat().shuffle(num_ej_entrenamiento).batch(TAMANO_LOTE)
datos_pruebas=datos_pruebas.batch(TAMANO_LOTE)

import math
# entrenar

historial = modelo.fit( datos_entrenamiento, epochs=5, steps_per_epoch=math.ceil(num_ej_entrenamiento/TAMANO_LOTE) )

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 76s 36ms/step - accuracy: 0.8212 - loss: 0.5520
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 64s 34ms/step - accuracy: 0.9622 - loss: 0.1271
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 64s 34ms/step - accuracy: 0.9735 - loss: 0.0894
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 65s 35ms/step - accuracy: 0.9758 - loss: 0.0769
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 64s 34ms/step - accuracy: 0.9808 - loss: 0.0626


In [34]:
# Guardar en formato SavedModel
modelo.export('numeros_tfjs')


Saved artifact at 'numeros_tfjs'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor_61')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  134208653523088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134208653518672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134208653514832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134208653517328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134208653515792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134208653515600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134208653515984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134208653516368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134208653521744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134208653516176: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [35]:
# Instalar librería de conversión
!pip install tensorflowjs


In [36]:
# Crear carpeta de salida
!mkdir carpeta_salida

mkdir: cannot create directory ‘carpeta_salida’: File exists


In [37]:
# Convertir a TensorFlow.js GraphModel
!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model numeros_tfjs carpeta_salida

2026-02-02 20:51:05.714931: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770065465.761754   34945 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770065465.778098   34945 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770065465.801708   34945 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770065465.801812   34945 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770065465.801820   34945 computation_placer.cc:177] computation placer alr

In [38]:
# Confirmar que se generaron los archivos
!ls carpeta_salida

group1-shard1of1.bin  model.json
